Please compile the codes for class and function definition before running the codes for query

In [3]:
import os
from zipfile import ZipFile
import nltk
nltk.download('punkt')
nltk.download("stopwords")
import re
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))
class Positional_Index:
  def __init__(self):
    self.index_dict=dict()#dictionary to store positional index
    self.token_list=list()
    self.freq_dict=dict()#dictionary to frequency of a word
    self.re=re
    self.nltk=nltk
    self.nltk.download("punkt")
    self.nltk.download("stopwords")
    self.stop_words=list(stopwords.words("english"))

  def make_tok_list(self):
    if not os.path.isdir('temp_dir1'):
      os.mkdir('temp_dir1')
    with ZipFile("/content/CSE508_Winter2023_Dataset2_preprocessed.zip", 'r') as zObject:    # Extracting all the members of the zip 
    # into a specific location.
      zObject.extractall(path="temp_dir1")
    dir_path='/content/temp_dir1/content/temp_dir/CSE508_Winter2023_Dataset'
    files_list=os.listdir(dir_path)
    file_tok_list=list()
    self.mapping_dict=dict()# mapping to docID to doc name
    k=0
    for fl in files_list:# storing tokens w.rt. a doc
      fl_path=os.path.join(dir_path,fl)
      fobj=open(fl_path,'r')
      self.mapping_dict[k]=fl
      st=fobj.read()
      word_toks=st.split(',')
      self.token_list.append(word_toks)
      fobj.close()
      k=k+1

  def make_index(self):# make the index
    fin_token_list=set()
    #index_dict=dict()
    self.make_tok_list()
    for toks in self.token_list:
      fin_token_list=fin_token_list.union(set(toks))
    fin_token_list=list(fin_token_list)# get all the unique tokens present in the docs
    for tok in fin_token_list:
      self.index_dict[tok]=dict()
    for doc in range(len(self.token_list)):
      for pos in range(len(self.token_list[doc])):
        if doc not in self.index_dict[self.token_list[doc][pos]].keys():# first time a position from a document d is encountered
          self.index_dict[self.token_list[doc][pos]][doc]=[]
          self.index_dict[self.token_list[doc][pos]][doc].append(pos)# adding the position
        else:
          self.index_dict[self.token_list[doc][pos]][doc].append(pos)# adding the position in general sense
      #index_dict[self.token_list[doc][pos]]
    #return self.index_dict
  def preprocess_query(self,query):# preprocessing the query
    query = query.lower()
    words = self.re.sub('[^\w\s]',' ',query)
    tokens = self.nltk.word_tokenize(words)
    new_tokens = [t for t in tokens if t not in self.stop_words]
    words_list = [t for t in new_tokens if t!=' ']
    return words_list

  def process_query(self,query_text):
        q_toks=self.preprocess_query(query_text)# preprocessing our query into tokens and cleaning it
        #print(q_toks)
        pos_list_dict=dict()
        for i in range(0,len(q_toks),1):
          #print(i)
          if q_toks[i] in self.index_dict.keys():# chcking if the token is present
              for doc in self.index_dict[q_toks[i]].keys():
                #print(doc)
                if i==0 and (doc not in pos_list_dict.keys()):# for the first token, add the docs present with corresponding positions
                  pos_list=self.index_dict[q_toks[i]][doc]
                  pos_list_dict[doc]=set()
                  #print(pos_list)
                  pos_list_dict[doc]=pos_list_dict[doc].union(set(pos_list))# adding the positions of the docs using our current token 
                elif doc in pos_list_dict.keys():# if doc is already present
                  pos_list=self.index_dict[q_toks[i]][doc]
                  pos_list=[(pos-i)  for pos in pos_list]# decrement the positions to get aligned with the 1st tokens, as having the same positions will help here, maintains that tokens are consecutive 
                  #print(pos_list)
                  pos_list_dict[doc]=pos_list_dict[doc].intersection(set(pos_list))# intersection of common positions
              key_list_dict= dict()
              for pos in pos_list_dict.keys():# removing the docs containing the previous tokens but not the current one
                if pos in self.index_dict[q_toks[i]].keys() and len(pos_list_dict[pos])>0:
                  key_list_dict[pos]=pos_list_dict[pos]
              pos_list_dict=key_list_dict
              doc_name_list=[self.mapping_dict[pos] for pos in pos_list_dict.keys()]  
          else:
                return [],0,[]          
        return pos_list_dict,len(pos_list_dict.keys()),doc_name_list
    #except:
      #print("errors have occurred, most likely your phrase is not present in our database")
      #return [],0,[]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import os
from zipfile import ZipFile
import nltk
nltk.download('punkt')
nltk.download("stopwords")
import re 
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))
class Bigram_Index:
  def __init__(self):
    self.index_dict=dict()# dictionary to keep the inverted index
    self.token_list=list()# list containing tokens for each doc
    self.freq_dict=dict()# storing frequency of each term
    self.re=re
    self.nltk=nltk
    self.nltk.download("punkt")
    self.nltk.download("stopwords")
    self.stop_words=list(stopwords.words("english"))


  def make_bigram(self):
    if not(os.path.isdir('temp_dir1')):
      os.mkdir('temp_dir1')
    with ZipFile("/content/CSE508_Winter2023_Dataset2_preprocessed.zip", 'r') as zObject:    # Extracting all the members of the zip 
    # into a specific location.
      zObject.extractall(path="temp_dir1")
    dir_path='/content/temp_dir1/content/temp_dir/CSE508_Winter2023_Dataset'
    files_list=os.listdir(dir_path)
    file_tok_list=list()
    self.mapping_dict=dict()# dociD to doc name
    k=0
    for fl in files_list:
      fl_path=os.path.join(dir_path,fl)
      fobj=open(fl_path,'r')
      self.mapping_dict[k]=fl
      st=fobj.read()
      word_toks=list(nltk.bigrams(st.split(",")))# creating bigram tokens/terms at word level
      self.token_list.append(word_toks)
      fobj.close()
      k=k+1

  def make_index(self):
    fin_token_list=set()
    #index_dict=dict()
    self.make_bigram()
    for toks in self.token_list:
      fin_token_list=fin_token_list.union(set(toks))
    fin_token_list=list(fin_token_list)# creating list of unique biword/bigram terms
    #print(fin_token_list)
    for tok in fin_token_list:#initializing
      self.index_dict[' '.join(tok)]=list()
      self.freq_dict[' '.join(tok)]=0
    for doc in range(len(self.token_list)):
      #print(self.token_list[doc])
      for bg in self.token_list[doc]:
          #print(bg)
          if doc not in self.index_dict[' '.join(bg)]:#if document has not been yet stored for a bigram
            self.index_dict[' '.join(bg)].append(doc)
          self.freq_dict[' '.join(bg)]=self.freq_dict[' '.join(bg)]+1 # updating frequency
      #index_dict[self.token_list[doc][pos]]
    #return self.index_dict

  def preprocess_query(self,query):# preprocessing the query
    words = self.re.sub('[^\w\s]',' ',query)
    tokens = self.nltk.word_tokenize(words)
    new_tokens = [t for t in tokens if t not in self.stop_words]
    words_list = [t for t in new_tokens if t!=' ']
    return words_list

  def process_query(self,query_text):
    q_processed=self.preprocess_query(query_text)# get preprocessed tokens
    q_toks=list(nltk.bigrams(q_processed))# form bigrams of the query
    pos_list_set=set()# list to store the positions
    for i in range(len(q_toks)):
      if ' '.join(q_toks[i]) in self.index_dict.keys():
          if i==0:
            pos_list_set=pos_list_set.union(set(self.index_dict[' '.join(q_toks[i])]))# storing docs of 1st bigram
          else:
            pos_list_set=pos_list_set.intersection(set(self.index_dict[' '.join(q_toks[i])]))# only common docs survive the querying process
          doc_names_list=[self.mapping_dict[pos] for pos in pos_list_set]
          return pos_list_set,len(pos_list_set),doc_names_list
      else:
        return {},0,[]# if any bigram is missing, we return null set


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from dill import dump,load

If you are running the object_make function, you should be able to download the pickle files. Still we will be attaching our downloaded files for your reference as well. 

In [ ]:
def object_make():#creating the object for each index and pickling them
  pi=Positional_Index()
  pi.make_index()
  bi=Bigram_Index()
  bi.make_index()
  f3=open("pos_index_dill.pkl",'wb')
  dump(pi,f3)
  f3.close()
  f4=open("bigram_index_dill.pkl",'wb')
  dump(pi,f4)
  f4.close()
  files.download("bigram_index_dill.pkl")# keeping track of the pickle files
  files.download("pos_index_dill.pkl")

Make sure to upload the pickle files of bigram_index_dill.pkl and pos_index_dill.pkl. If the files are uploaded once, you do not need to do so. However, if you need to re-upload, please delete the files or current session. Else you need to change the file path for the same. 

In [ ]:
from google.colab import files
uploads=files.upload()

Saving CSE508_Winter2023_Dataset2_preprocessed.zip to CSE508_Winter2023_Dataset2_preprocessed.zip


In [ ]:
object_make()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# do not forget to run this code
def get_query_results(bi_index,pi_index):# taking indices as input
  n=int(input("enter number of queries:"))# user query
  for i in range(n):
    print("enter query number",str(i+1))
    query=input()
    pos_list,docs_num1,docs_names1=bi_index.process_query(query)
    pos_dict,docs_num2,docs_names2=pi_index.process_query(query)
    print("Number of documents retrieved for query ",str(i+1)," using bigram inverted index:",docs_num1)
    print("Names of documents retrieved for query ",str(i+1)," using bigram inverted index:",docs_names1)
    print("Number of documents retrieved for query ",str(i+1)," using positional index:",docs_num2)
    print("Names of documents retrieved for query ",str(i+1)," using positional index:",docs_names2)




Make sure to upload the pickle files of bigram_index_dill.pkl and pos_index_dill.pkl. If the files are uploaded once, you do not need to do so. However, if you need to re-upload, please delete the files or current session. Else you need to change the file path for the same. 

In [1]:
from google.colab import files
uploads=files.upload()

Saving bigram_index_dill.pkl to bigram_index_dill.pkl
Saving pos_index_dill.pkl to pos_index_dill.pkl


In [2]:
from dill import dump,load# this line should be executed, please do not forget

In [7]:
fp=open('/content/pos_index_dill.pkl','rb')
fb=open('/content/bigram_index_dill.pkl','rb')
pi=load(fp)
bi=load(fb)
get_query_results(bi,pi)# getting the query output
fp.close()
fb.close()

enter number of queries:2
enter query number 1
stressing heated wings
Number of documents retrieved for query  1  using bigram inverted index: 1
Names of documents retrieved for query  1  using bigram inverted index: ['cranfield0013']
Number of documents retrieved for query  1  using positional index: 1
Names of documents retrieved for query  1  using positional index: ['cranfield0013']
enter query number 2
distribution conical bodies hypersonic
Number of documents retrieved for query  2  using bigram inverted index: 1
Names of documents retrieved for query  2  using bigram inverted index: ['cranfield0019']
Number of documents retrieved for query  2  using positional index: 1
Names of documents retrieved for query  2  using positional index: ['cranfield0019']
